<a href="https://colab.research.google.com/github/YaninaK/anomaly-detection/blob/b1/notebooks/02_Anomalies_1_2_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Обнаружение аномалий в начислениях за тепловую энергию


## 2. Аномалии 1, 2 и 4.



### Примеры аномалий

Виды аномалий по показаниям приборов учёта тепловой энергии, которые необходимо выявлять (кроме объектов с видом энергопотребления ГВС (централ):

1. нулевые значения показаний за тепловую энергию в отопительный период (октябрь-апрель);

2. равные значения показаний в течение нескольких расчетных периодов;

3. снижение/рост показаний в отдельные месяцы по сравнению с показаниями за предыдущие периоды по данному объекту (с учётом фактической температуры наружного воздуха и количества отопительных дней в месяце);

4. аномально низкое/высокое (отклонение более 25%) потребление объекта в конкретном месяце по сравнению с аналогичными объектами (только для типов объекта «Многоквартирный дом») по критериям:
  - год постройки (по группам до 1958 г., 1959-1989 гг., 1990-2000 гг., 2001-2010 гг., 2011-2024 гг.),
  - этажность (по группам 1-2 этажа, 3-4 этажа, 5-9 этажей,10-12 этажей, 13 и более этажей),
  - площадь (±10%),
  - наличие ГВС ИТП (горячей воды, учитываемой тем же прибором).



In [1]:
initiate = True
if initiate:
  !git init -q
  !git clone -b b1  https://github.com/YaninaK/anomaly-detection.git -q

  from google.colab import drive
  drive.mount('/content/drive')

  !unzip -u -q /content/drive/MyDrive/ML_projects/08_anomaly_detection/data/01_raw/task#3.zip -d /content/anomaly-detection/data/01_raw

%cd /content/anomaly-detection/notebooks

Mounted at /content/drive
/content/anomaly-detection/notebooks


In [2]:
import os
import sys

sys.path.append(os.getcwd())
sys.path.append(os.path.join(os.getcwd(), "..", "src", "anomaly_detection"))

In [3]:
import datetime

import numpy as np
import pandas as pd

from data.make_dataset import load_data
from data.preprocess import Preprocess
from features.data_sequence import generate_data_sequence
from features.missing_records import (
    select_missing_records, select_uninvoiced_objects, select_nonunique_objects
)
from features.duplicated import get_equal_values
from features.grouping import Grouping
from features.period_outliers import get_outlers

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
PATH = "/content/anomaly-detection/"

## 1. Чтение данных

In [6]:
folder_path = '../data/01_raw/'

In [7]:
regenerate = True
data, temperature, buildings = load_data(folder_path, regenerate, path=PATH)

100%|██████████| 24/24 [00:12<00:00,  1.90it/s]


## 2. Подготовка данных

In [8]:
preprocess = Preprocess()
data, buildings = preprocess.fit_transform(data, buildings)

data.shape

(75385, 10)

In [9]:
df = generate_data_sequence(data)
df.shape

print(f"df.shape = {df.shape}\n")
df.sample(2)

df.shape = (4628, 24)



,,,,,2021-07-01,2021-08-01,2021-09-01,2021-10-01,2021-11-01,2021-12-01,2022-01-01,2022-02-01,2022-03-01,2022-04-01,...,2022-09-01,2022-10-01,2022-11-01,2022-12-01,2023-01-01,2023-02-01,2023-03-01,2023-04-01,2023-05-01,2023-06-01
Адрес объекта,Тип объекта,№ ОДПУ,Вид энерг-а ГВС,Адрес объекта 2,,,,,,,,,,,,,,,,,,,,,
"г Уфа, ул. Юрия Гагарина, д.43",Многоквартирный дом,102399,0,"г Уфа, ул. Юрия Гагарина, д.43",NaN,NaN,3.357,53.530,85.957,113.883,140.249,108.9680,100.996,71.758,...,0.000,42.994,87.404,130.2303,131.149,112.481559,86.245,61.993,NaN,NaN
"г Уфа, ул. Пархоменко, д.106 корп.2",Многоквартирный дом,93051,1,"г Уфа, ул. Пархоменко, д.106 корп.2",22.343,10.137,23.890,93.593,137.800,168.120,200.717,155.8157,162.087,132.436,...,27.674,75.139,116.639,169.6190,184.585,154.353000,127.457,117.602,25.343,21.31


## 3. Аномалии



### 3.1 Нулевые значения показаний за тепловую энергию в отопительный период (октябрь-апрель)

#### 3.1.1 Пропуски в данных по текущему потреблению

In [10]:
save=True
all_periods = False
missing_records = select_missing_records(df, all_periods, save, path=PATH)

print(f"missing_records.shape = {missing_records.shape}\n")
missing_records.sample(2)

missing_records.shape = (1053, 14)



,,,,,2021-10-01 00:00:00,2021-11-01 00:00:00,2021-12-01 00:00:00,2022-01-01 00:00:00,2022-02-01 00:00:00,2022-03-01 00:00:00,2022-04-01 00:00:00,2022-10-01 00:00:00,2022-11-01 00:00:00,2022-12-01 00:00:00,2023-01-01 00:00:00,2023-02-01 00:00:00,2023-03-01 00:00:00,2023-04-01 00:00:00
Адрес объекта,Тип объекта,№ ОДПУ,Вид энерг-а ГВС,Адрес объекта 2,,,,,,,,,,,,,,
"г Уфа, ул. Дмитрия Донского, д.1 корп.2",Другое строение,218670,0,"г Уфа, ул. Дмитрия Донского, д.1 корп.2",NaN,NaN,8.076,8.915,7.976,6.951,4.237,0.000,4.376,8.571,9.480,7.986,5.308,2.513
"г Уфа, ул. Летчиков, д.2",Другое строение,"127906, 127899",0,"г Уфа, ул. Летчиков, д.2",NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.253,132.653,234.305,263.443,229.822,159.236,122.811


#### 3.1.2 Объекты без данных по текущему потреблению

In [11]:
save = True

uninvoiced_objects = select_uninvoiced_objects(df, buildings, save, path=PATH)

print(f"uninvoiced_objects.shape = {uninvoiced_objects.shape}")
uninvoiced_objects.sample(2)

uninvoiced_objects.shape = (1650, 6)


,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта,Адрес объекта 2
1616,"г Уфа, ул. Энтузиастов, д.1",Другое строение,1,1976-01-01,0.01,"г Уфа, ул. Энтузиастов, д.1"
398,"г Уфа, ул. Герцена, д.56",Частный дом,1,1950-01-01,101.00,"г Уфа, ул. Герцена, д.56"


In [12]:
pd.concat(
    [
        uninvoiced_objects["Тип Объекта"].value_counts(),
        uninvoiced_objects["Тип Объекта"].value_counts(normalize=True)
    ], axis=1
).head(10)

,count,proportion
Тип Объекта,,
Многоквартирный дом,752,0.455758
Другое строение,652,0.395152
Частный дом,138,0.083636
"Административные здания, конторы",45,0.027273
"Учебное заведение, комбинат, центр",13,0.007879
Школы и ВУЗ,13,0.007879
Гаражи,6,0.003636
"Жилое здание (Гостиница, Общежитие)",5,0.003030
Производственный объект,4,0.002424


In [13]:
cond = uninvoiced_objects["Тип Объекта"] == "Многоквартирный дом"
s = uninvoiced_objects[cond]['Общая площадь объекта'].sum() / 1e06

print(f"У {cond.sum()} объектов типа Многоквартирный дом общей площадью{s: 0.1f} млн. нет данных о выствленных счетах.")

У 752 объектов типа Многоквартирный дом общей площадью 1.1 млн. нет данных о выствленных счетах.


* У 1650 объектов нет данных об учете потребления теплоэнергии. Из них 752 объекта (46%) общей площадью 1.1 млн. приходится на тип ```Многоквартирный дом```, около 40% - на тип ```Другое строение```, около 8.3% - на тип ```Частный дом```.


#### 3.1.3 Неуникальные адреса объектов.

In [14]:
save = True
nonunique = select_nonunique_objects(buildings, save, path=PATH)

print(f"Всего неуникальных объектов: {nonunique.shape[0]}\n")
nonunique.head(6)

Всего неуникальных объектов: 145



,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта,Адрес объекта 2
122,"г Уфа, б-р. Тухвата Янаби, д.34",Другое строение,1,NaT,0.01,"г Уфа, б-р. Тухвата Янаби, д.34"
123,"г Уфа, б-р. Тухвата Янаби, д.34",Другое строение,2,NaT,0.01,"г Уфа, б-р. Тухвата Янаби, д.34"
412,"г Уфа, пр-кт Октября, д.153",Другое строение,0,NaT,0.01,"г Уфа, пр-кт Октября, д.153"
413,"г Уфа, пр-кт Октября, д.153",Другое строение,3,NaT,0.01,"г Уфа, пр-кт Октября, д.153"
487,"г Уфа, пр-кт Октября, д.31",Другое строение,0,1970-01-01,0.01,"г Уфа, пр-кт Октября, д.31"
488,"г Уфа, пр-кт Октября, д.31",Другое строение,5,1970-01-01,0.01,"г Уфа, пр-кт Октября, д.31"


In [15]:
pd.concat(
    [
      nonunique["Тип Объекта"].value_counts(),
      nonunique["Тип Объекта"].value_counts(True),
      nonunique.groupby(["Тип Объекта"])["Адрес объекта"].agg(lambda x: x.nunique())
    ], axis=1
)

,count,proportion,Адрес объекта
Тип Объекта,,,
Другое строение,116,0.800000,51
Многоквартирный дом,14,0.096552,7
Частный дом,12,0.082759,4
"Учебное заведение, комбинат, центр",3,0.020690,1


1. Типы объектов ```Другое строение``` и ```Частный дом``` оставлены без корректировок:
  * Уникальный адрес объекта нужен для привязки площади объекта и даты постройки к данным учета потребления теплоэнергии. У типа ```Другое строение``` - в большинстве случаев эти данные отстутствуют, соответственно этот блок информации не имеет смысла корректировать.

  * Информации об неуникальных объектах типа ```Частный дом``` нет в данных о потреблении теплоэнергии - этот блок также не имеет смысла корректировать.

2. Блок ```Многоквартирный дом``` скорректирован:
  * Адреса многоквартирных домов сделаны уникальными в признаке ```Адрес объекта 2``` через сопоставление с данными об учете теплоэнергии.
  * На трех объектах ниже счета не выставляются, для них в признаке ```Адрес объекта 2``` к адресу добавлено слово ```extra``` :
    * ```г Уфа, ул. Вологодская, д.20```,
    * ```г Уфа, ул. Интернациональная, д.113```,
    * ```г Уфа, ул. Нежинская, д.6```.
3. Блок ```Учебное заведение, комбинат, центр``` скорректирован:
  * Адреса сделаны уникальными в признаке ```Адрес объекта 2``` через сопоставление с данными об учете теплоэнергии и проставление ``№ ОДПУ`` в адрес.

In [16]:
nonunique_2 = buildings[
    buildings.duplicated(subset=["Адрес объекта 2", "Тип Объекта"], keep=False)
]
pd.concat(
    [
      nonunique_2["Тип Объекта"].value_counts(),
      nonunique_2["Тип Объекта"].value_counts(True),
      nonunique_2.groupby(["Тип Объекта"])["Адрес объекта 2"].agg(lambda x: x.nunique())
    ], axis=1
)

,count,proportion,Адрес объекта 2
Тип Объекта,,,
Другое строение,115,0.905512,51
Частный дом,12,0.094488,4


* Если использовать ```Адрес объекта 2```, неуникальные адреса в типах  ```Многоквартирный дом``` и ```Учебное заведение, комбинат, центр``` появляться не будут.

## 3.2 Равные значения показаний в течение нескольких расчетных периодов

In [17]:
save = True
equal_values = get_equal_values(data, save, path=PATH)

print(f"Число записей с равными значениями показаний: {equal_values.shape[0]}")
print(f"Число адресов с равными значениями показаний: {equal_values['Адрес объекта'].nunique()}\n")
equal_values.tail(10)

Число записей с равными значениями показаний: 242
Число адресов с равными значениями показаний: 94



,index,Подразделение,№ ОДПУ,Вид энерг-а ГВС,Адрес объекта,Тип объекта,Дата текущего показания,"Текущее потребление, Гкал",Период потребления
65236,4145,Уфа,1524327,None,"г Уфа, ул. Энтузиастов, д.14, Подобъект №984513",Многоквартирный дом,2022-03-01,1.000,2022-02-01
57777,2746,Уфа,1524327,None,"г Уфа, ул. Энтузиастов, д.14, Подобъект №984513",Многоквартирный дом,2022-04-01,1.000,2022-03-01
19082,3738,Уфа,"00535101751, 00533101751",None,"г Уфа, ул. Энтузиастов, д.16, Подобъект №984098",Многоквартирный дом,2021-12-01,2.500,2021-11-01
46913,4292,Уфа,"00535101751, 00533101751",None,"г Уфа, ул. Энтузиастов, д.16, Подобъект №984098",Многоквартирный дом,2022-12-01,2.500,2022-11-01
13060,4294,Уфа,65896908,None,"г Уфа, ул. Энтузиастов, д.16, Подобъект №984816",Многоквартирный дом,2023-04-01,2.519,2023-03-01
42496,4285,Уфа,65896908,None,"г Уфа, ул. Энтузиастов, д.16, Подобъект №984816",Многоквартирный дом,2023-05-01,2.519,2023-04-01
24894,4069,Уфа,101433,None,"г Уфа, ул. Юрия Гагарина, д.20",Другое строение,2021-11-01,11.009,2021-10-01
54942,4231,Уфа,101433,None,"г Уфа, ул. Юрия Гагарина, д.20",Другое строение,2022-11-01,11.009,2022-10-01
36090,3524,Уфа,1981,None,"р-н Уфимский, д Лекаревка",Другое строение,2022-01-01,210.000,2021-12-01
57663,2632,Уфа,1981,None,"р-н Уфимский, д Лекаревка",Другое строение,2022-04-01,210.000,2022-03-01


## 3.3. Аномально низкое/высокое (отклонение более 25%) потребление объекта в конкретном месяце по сравнению с аналогичными объектами

(только для типов объекта «Многоквартирный дом») по критериям:
  - год постройки (по группам до 1958 г., 1959-1989 гг., 1990-2000 гг., 2001-2010 гг., 2011-2024 гг.),
  - этажность (по группам 1-2 этажа, 3-4 этажа, 5-9 этажей,10-12 этажей, 13 и более этажей),
  - площадь (±10%),
  - наличие ГВС ИТП (горячей воды, учитываемой тем же прибором).

In [18]:
grouping = Grouping()
df_grouped, dropped = grouping.fit_transform(df, buildings)

print(f"df_grouped.shape = {df_grouped.shape}\n")
df_grouped.sample(2)

df_grouped.shape = (3204, 35)



,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта,Адрес объекта 2,Группа этажность объекта,Улица,Дата постройки 2,Группа год постройки,...,2022-09-01 00:00:00,2022-10-01 00:00:00,2022-11-01 00:00:00,2022-12-01 00:00:00,2023-01-01 00:00:00,2023-02-01 00:00:00,2023-03-01 00:00:00,2023-04-01 00:00:00,2023-05-01 00:00:00,2023-06-01 00:00:00
956,"г Уфа, ул. Борисоглебская, д.12",Многоквартирный дом,5.0,1971-01-01,3029.7,"г Уфа, ул. Борисоглебская, д.12",5-9 этажей,ул. Борисоглебская,1971-01-01,1959-1989 гг.,...,NaN,78.0610,79.879,113.571,117.013,102.253,78.245,60.424,NaN,NaN
1037,"г Уфа, ул. Вологодская, д.27 корп.1",Многоквартирный дом,9.0,1983-01-01,7530.6,"г Уфа, ул. Вологодская, д.27 корп.1",5-9 этажей,ул. Вологодская,1983-01-01,1959-1989 гг.,...,NaN,59.7437,143.135,226.006,230.737,180.928,130.754,98.093,NaN,NaN


In [19]:
dropped

,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта,Адрес объекта 2,Группа этажность объекта,Улица,Дата постройки 2,Группа год постройки,...,2022-09-01 00:00:00,2022-10-01 00:00:00,2022-11-01 00:00:00,2022-12-01 00:00:00,2023-01-01 00:00:00,2023-02-01 00:00:00,2023-03-01 00:00:00,2023-04-01 00:00:00,2023-05-01 00:00:00,2023-06-01 00:00:00
1432,"г Уфа, ул. Кирова, д.95",Многоквартирный дом,1.0,1971-01-01,0.01,"г Уфа, ул. Кирова, д.95, Подобъект №46590",1-2 этажа,ул. Кирова,1971-01-01,1959-1989 гг.,...,NaN,6.623,18.004,14.127,29.448,25.116,18.209,11.972,NaN,NaN


Удалены объекты с неуказанной общей площадью:

1. Нет информации ```Этажность объекта```, ```Дата постройки```, ```Общая площадь объекта``` по многоквартирному дому по адресу ```г Уфа, ул. Уфимское шоссе, д.4```.

2. По адресу ```г Уфа, ул. Кирова, д.95``` значатся 2 многоквартирных дома: 5 этажей и 1 этаж. По одноэтажному дому (```Подобъект №46590```) нет информации по общей площади объекта.

In [20]:
df_grouped.groupby(
    ["Группа год постройки", "Группа этажность объекта", "Вид энерг-а ГВС"]
)["Общая площадь объекта"].count()

Группа год постройки  Группа этажность объекта  Вид энерг-а ГВС
до 1958 г             1-2 этажа                 0                     7
                                                1                     3
                      3-4 этажа                 0                    92
                                                1                    79
                      5-9 этажей                0                    79
                                                1                    34
                      10-12 этажей              0                     1
                                                1                     0
                      13 и более этажей         0                     0
                                                1                     2
1959-1989 гг.         1-2 этажа                 0                    11
                                                1                     0
                      3-4 этажа                 0                    35
                                                1                    20
                      5-9 этажей                0                  1367
                                                1                   406
                      10-12 этажей              0                   168
                                                1                    11
                      13 и более этажей         0                    14
                                                1                     4
1990-2000 гг.         1-2 этажа                 0                     0
                                                1                     0
                      3-4 этажа                 0                     1
                                                1                     1
                      5-9 этажей                0                   205
                                                1                    34
                      10-12 этажей              0                   100
                                                1                    19
                      13 и более этажей         0                    20
                                                1                    12
2001-2010 гг.         1-2 этажа                 0                     0
                                                1                     0
                      3-4 этажа                 0                     1
                                                1                     0
                      5-9 этажей                0                    48
                                                1                    21
                      10-12 этажей              0                    61
                                                1                    34
                      13 и более этажей         0                    56
                                                1                    45
2011-2024 гг.         1-2 этажа                 0                     0
                                                1                     0
                      3-4 этажа                 0                     0
                                                1                     0
                      5-9 этажей                0                     1
                                                1                     6
                      10-12 этажей              0                     5
                                                1                    26
                      13 и более этажей         0                    28
                                                1                   146
Name: Общая площадь объекта, dtype: int64

* Наибольшее число объектов - 1367 в группе
  * год постройки: 1959-1989 гг.
  * этажность объекта: 5-9 этажей
  * Вид энерг-а ГВС: 0

In [21]:
save=True
n_periods = 24
threshold = 0.25

outliers_addr, consumption_mask = get_outlers(
    df_grouped, threshold=threshold, n_periods=n_periods,  save=save, path=PATH
)
print(f"Aномально низкое/высокое (отклонение более {threshold:.0%}) потребление:")
for period in outliers_addr:
  n_underconsumption_addr = len(outliers_addr[period]["underconsumption"])
  n_overconsumption_addr = len(outliers_addr[period]["overconsumption"])
  print(f"{period: %Y-%m}:  {n_underconsumption_addr}\t{n_overconsumption_addr}")

Aномально низкое/высокое (отклонение более 25%) потребление:
 2021-07:  154	202
 2021-08:  182	193
 2021-09:  207	253
 2021-10:  361	579
 2021-11:  211	358
 2021-12:  207	281
 2022-01:  213	268
 2022-02:  194	274
 2022-03:  221	285
 2022-04:  286	388
 2022-05:  151	193
 2022-06:  135	192
 2022-07:  153	202
 2022-08:  175	213
 2022-09:  133	169
 2022-10:  475	770
 2022-11:  217	339
 2022-12:  205	289
 2023-01:  206	272
 2023-02:  182	277
 2023-03:  220	296
 2023-04:  371	473
 2023-05:  147	204
 2023-06:  168	225


In [22]:
print(f"consumption_mask.shape = {consumption_mask.shape}\n")
consumption_mask.sample(2)

consumption_mask.shape = (3204, 35)



,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта,Адрес объекта 2,Группа этажность объекта,Улица,Дата постройки 2,Группа год постройки,...,2022-09-01 00:00:00,2022-10-01 00:00:00,2022-11-01 00:00:00,2022-12-01 00:00:00,2023-01-01 00:00:00,2023-02-01 00:00:00,2023-03-01 00:00:00,2023-04-01 00:00:00,2023-05-01 00:00:00,2023-06-01 00:00:00
1224,"г Уфа, ул. Запотоцкого, д.12",Многоквартирный дом,10.0,2008-01-01,5411.9,"г Уфа, ул. Запотоцкого, д.12",10-12 этажей,ул. Запотоцкого,2008-01-01,2001-2010 гг.,...,NaN,1.0,2.0,2.0,2.0,2.0,2.0,1.0,NaN,NaN
1029,"г Уфа, ул. Вологодская, д.20",Многоквартирный дом,12.0,1978-01-01,3681.4,"г Уфа, ул. Вологодская, д.20",10-12 этажей,ул. Вологодская,1978-01-01,1959-1989 гг.,...,NaN,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,NaN


* Объекты, на которых встречалось аномально низкое потребление - на 25% ниже медианного в рамках группы:
  - год постройки (по группам до 1958 г., 1959-1989 гг., 1990-2000 гг., 2001-2010 гг., 2011-2024 гг.),
  - этажность (по группам 1-2 этажа, 3-4 этажа, 5-9 этажей,10-12 этажей, 13 и более этажей),
  - площадь (±10%),
  - наличие ГВС ИТП (горячей воды, учитываемой тем же прибором).
* В ```consumption_mask``` помечены цифрой 1.

In [23]:
underconsumption = df_grouped[(consumption_mask.iloc[:, -n_periods:] == 1).sum(axis=1) > 0]

print(f"underconsumption.shape = {underconsumption.shape}\n")
underconsumption.sample(2)

underconsumption.shape = (1274, 35)



,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта,Адрес объекта 2,Группа этажность объекта,Улица,Дата постройки 2,Группа год постройки,...,2022-09-01 00:00:00,2022-10-01 00:00:00,2022-11-01 00:00:00,2022-12-01 00:00:00,2023-01-01 00:00:00,2023-02-01 00:00:00,2023-03-01 00:00:00,2023-04-01 00:00:00,2023-05-01 00:00:00,2023-06-01 00:00:00
2384,"г Уфа, ул. Революционная, д.31",Многоквартирный дом,9.0,1984-01-01,8624.3,"г Уфа, ул. Революционная, д.31",5-9 этажей,ул. Революционная,1984-01-01,1959-1989 гг.,...,NaN,138.609,159.144,154.0100,NaN,123.963,133.545,96.2600,NaN,NaN
3093,"г Уфа, ул. Энтузиастов, д.13",Многоквартирный дом,20.0,2018-10-01,36449.1,"г Уфа, ул. Энтузиастов, д.13",13 и более этажей,ул. Энтузиастов,2018-10-01,2011-2024 гг.,...,155.44,441.620,731.817,1116.8024,1161.31,935.161,783.220,518.8358,140.65,125.5844


* Объекты, на которых встречалось аномально высокое потребление - на 25% выше медианного в рамках группы:
  - год постройки (по группам до 1958 г., 1959-1989 гг., 1990-2000 гг., 2001-2010 гг., 2011-2024 гг.),
  - этажность (по группам 1-2 этажа, 3-4 этажа, 5-9 этажей,10-12 этажей, 13 и более этажей),
  - площадь (±10%),
  - наличие ГВС ИТП (горячей воды, учитываемой тем же прибором).
* В ```consumption_mask``` помечены цифрой 3.

In [24]:
overconsumption = df_grouped[(consumption_mask.iloc[:, -n_periods:] == 3).sum(axis=1) > 0]

print(f"overconsumption.shape = {overconsumption.shape}\n")
overconsumption.sample(2)

overconsumption.shape = (1662, 35)



,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта,Адрес объекта 2,Группа этажность объекта,Улица,Дата постройки 2,Группа год постройки,...,2022-09-01 00:00:00,2022-10-01 00:00:00,2022-11-01 00:00:00,2022-12-01 00:00:00,2023-01-01 00:00:00,2023-02-01 00:00:00,2023-03-01 00:00:00,2023-04-01 00:00:00,2023-05-01 00:00:00,2023-06-01 00:00:00
964,"г Уфа, ул. Борисоглебская, д.28",Многоквартирный дом,5.0,1967-01-01,3197.3,"г Уфа, ул. Борисоглебская, д.28",5-9 этажей,ул. Борисоглебская,1967-01-01,1959-1989 гг.,...,NaN,46.514,69.077,100.230,107.220,92.919,69.855,54.428,NaN,NaN
1300,"г Уфа, ул. Интернациональная, д.193 корп.2",Многоквартирный дом,9.0,1999-01-01,3839.7,"г Уфа, ул. Интернациональная, д.193 корп.2",5-9 этажей,ул. Интернациональная,1999-01-01,1990-2000 гг.,...,NaN,91.137,112.426,160.283,165.206,139.780,103.760,68.991,NaN,NaN


* Объекты с потреблением +/- 25% от медианного потребления в рамках группы (описаны выше).
* В ```consumption_mask``` помечены цифрой 2.

In [25]:
normal = df_grouped[(consumption_mask.iloc[:, -n_periods:] == 2).sum(axis=1) > 0]

print(f"normal consumption.shape = {normal.shape}\n")
normal.sample(2)

normal consumption.shape = (3014, 35)



,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта,Адрес объекта 2,Группа этажность объекта,Улица,Дата постройки 2,Группа год постройки,...,2022-09-01 00:00:00,2022-10-01 00:00:00,2022-11-01 00:00:00,2022-12-01 00:00:00,2023-01-01 00:00:00,2023-02-01 00:00:00,2023-03-01 00:00:00,2023-04-01 00:00:00,2023-05-01 00:00:00,2023-06-01 00:00:00
147,"г Уфа, пр-кт Октября, д.10",Многоквартирный дом,5.0,1960-01-01,3778.5,"г Уфа, пр-кт Октября, д.10",5-9 этажей,пр-кт Октября,1960-01-01,1959-1989 гг.,...,NaN,38.641,66.03,97.516,98.671,82.911,66.198,57.837,NaN,NaN
561,"г Уфа, ул. 50-летия Октября, д.26",Многоквартирный дом,5.0,1966-01-01,3372.6,"г Уфа, ул. 50-летия Октября, д.26",5-9 этажей,ул. 50-летия Октября,1966-01-01,1959-1989 гг.,...,NaN,45.341,71.10,102.551,104.554,87.633,69.251,50.479,NaN,NaN
